# Important Variable

In [34]:
# !jupyter nbconvert --to script model_build.ipynb

import csv
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import neurokit2 as nk
from scipy.signal import butter, filtfilt
from scipy.signal import resample

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from scipy.stats import pearsonr, spearmanr, linregress
import random
import tensorflow as tf
from tensorflow.python.client import device_lib
import matplotlib.colors as mcolors

[NbConvertApp] Converting notebook model_build.ipynb to script
[NbConvertApp] Writing 42736 bytes to model_build.py


In [2]:
print(tf.__version__)

2.19.0


In [3]:
tf.config.experimental.list_physical_devices('GPU')
device_lib.list_local_devices()

W0000 00:00:1760991168.734105   11042 gpu_device.cc:2430] TensorFlow was not built with CUDA kernel binaries compatible with compute capability 12.0. CUDA kernels will be jit-compiled from PTX, which could take 30 minutes or longer.
W0000 00:00:1760991168.740665   11042 gpu_device.cc:2430] TensorFlow was not built with CUDA kernel binaries compatible with compute capability 12.0. CUDA kernels will be jit-compiled from PTX, which could take 30 minutes or longer.
I0000 00:00:1760991168.948262   11042 gpu_device.cc:2019] Created device /device:GPU:0 with 13121 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 5070 Ti, pci bus id: 0000:01:00.0, compute capability: 12.0


[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 11164436025408017048
 xla_global_id: -1,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 13758365696
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 16388617600126776170
 physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 5070 Ti, pci bus id: 0000:01:00.0, compute capability: 12.0"
 xla_global_id: 416903419]

In [35]:
MODEL_NAME = '3_inputs_model'
RUN = 'run1_' + MODEL_NAME+ '/'

CURRENT_DIR = os.getcwd() + '/..'
DATA_DIR = CURRENT_DIR + '/dataset/'
MASTER_OUTPUT_DIR = CURRENT_DIR + '/output/'
OUTPUT_DIR = MASTER_OUTPUT_DIR + RUN
print(DATA_DIR)
print(OUTPUT_DIR)

/mnt/e/fantasia_respiration/git_repo/ECG-based-respiratory-predictions/notebooks/../dataset/
/mnt/e/fantasia_respiration/git_repo/ECG-based-respiratory-predictions/notebooks/../output/run1_3_inputs_model/


In [5]:
#data_df[data_df['subject_id'] != 'f2y08']['snr'].describe()

# Load data

In [6]:
data_df = pd.read_pickle(DATA_DIR + 'fantasia_dataset_preprocessing.plk')
print("Before filter:", data_df.shape)
data_df

Before filter: (8339, 17)


,subject_id,chunk_id,ecg,resp,Mean_Respiration_Rate,Total_Area_Peak_to_Trough,Total_Area_Trough_to_Peak,Mean_latency_Trough_to_Peak,Mean_latency_Peak_to_Trough,hr_min,hr_mean,hr_max,hrv,snr,Total_peaks,Total_trough,Breath_rate_variability
0,f1o01,0,"[8.46, 8.592, 8.352, 8.152, 8.34, 8.528, 8.312...","[8.488, 8.452, 8.488, 8.496, 8.508, 8.476, 8.4...",14.505123,0.562117,1.169217,1.690667,2.544000,59.055118,62.032517,66.371681,16.625697,5.385646,6.0,6.0,310.321205
1,f1o01,1,"[8.316, 8.58, 8.664, 8.428, 8.264, 8.444, 8.58...","[8.116, 8.124, 8.092, 8.088, 8.06, 8.064, 8.07...",15.780254,1.146417,0.315799,1.525143,2.221333,58.593750,62.245654,66.079295,16.559194,5.984316,7.0,7.0,193.791641
2,f1o01,2,"[8.088, 8.088, 8.26, 8.304, 8.144, 8.072, 8.2,...","[8.656, 8.624, 8.608, 8.596, 8.648, 8.652, 8.6...",13.284661,0.353985,0.445506,1.772000,2.860000,57.692308,59.618603,61.728395,8.315218,6.507062,6.0,6.0,373.388872
3,f1o01,3,"[8.152, 8.02, 8.036, 8.208, 8.208, 8.128, 8.12...","[8.672, 8.684, 8.648, 8.632, 8.628, 8.672, 8.6...",14.913591,0.688938,1.089432,1.680000,2.391000,32.894737,62.955495,69.124424,261.545152,6.893892,5.0,5.0,1063.164772
4,f1o01,4,"[8.068, 8.004, 7.864, 7.9, 8.048, 7.996, 7.848...","[8.548, 8.524, 8.508, 8.528, 8.568, 8.552, 8.5...",16.444328,0.983089,0.837500,1.479200,2.752000,57.251908,62.337664,67.264574,14.326248,5.866205,5.0,5.0,1472.451584
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9471,f2y10,234,"[-0.01708984375, -0.01220703125, -0.01953125, ...","[0.54443359375, 0.5419921875, 0.5517578125, 0....",15.151441,0.219147,0.206771,1.567200,2.387000,49.180328,56.770476,62.761506,542.688370,6.556642,5.0,5.0,169.671251
9472,f2y10,235,"[-0.1220703125, -0.23681640625, -0.25390625, -...","[-0.17333984375, -0.1708984375, -0.17333984375...",10.984443,0.493156,0.759228,2.105000,3.357333,54.347826,58.418115,64.935065,47.516079,3.000940,4.0,4.0,125.795469
9473,f2y10,236,"[0.1025390625, 0.07568359375, 0.0439453125, 0....","[-0.185546875, -0.185546875, -0.1806640625, -0...",14.860658,0.272106,0.304594,1.684667,2.696000,50.000000,56.347270,64.655172,63.884896,7.360403,6.0,6.0,211.406007
9474,f2y10,237,"[-0.1123046875, -0.107421875, -0.10986328125, ...","[0.0830078125, 0.087890625, 0.0927734375, 0.09...",11.101286,0.637757,1.059193,2.117000,3.465333,50.335570,58.717126,69.444444,81.028010,3.194175,4.0,4.0,551.026769


In [7]:
data_df = data_df[(data_df['snr'] >= 4) & (data_df['Breath_rate_variability'] <= 1700)]
print("After filter:", data_df.shape)
data_df

After filter: (8105, 17)


,subject_id,chunk_id,ecg,resp,Mean_Respiration_Rate,Total_Area_Peak_to_Trough,Total_Area_Trough_to_Peak,Mean_latency_Trough_to_Peak,Mean_latency_Peak_to_Trough,hr_min,hr_mean,hr_max,hrv,snr,Total_peaks,Total_trough,Breath_rate_variability
0,f1o01,0,"[8.46, 8.592, 8.352, 8.152, 8.34, 8.528, 8.312...","[8.488, 8.452, 8.488, 8.496, 8.508, 8.476, 8.4...",14.505123,0.562117,1.169217,1.690667,2.544000,59.055118,62.032517,66.371681,16.625697,5.385646,6.0,6.0,310.321205
1,f1o01,1,"[8.316, 8.58, 8.664, 8.428, 8.264, 8.444, 8.58...","[8.116, 8.124, 8.092, 8.088, 8.06, 8.064, 8.07...",15.780254,1.146417,0.315799,1.525143,2.221333,58.593750,62.245654,66.079295,16.559194,5.984316,7.0,7.0,193.791641
2,f1o01,2,"[8.088, 8.088, 8.26, 8.304, 8.144, 8.072, 8.2,...","[8.656, 8.624, 8.608, 8.596, 8.648, 8.652, 8.6...",13.284661,0.353985,0.445506,1.772000,2.860000,57.692308,59.618603,61.728395,8.315218,6.507062,6.0,6.0,373.388872
3,f1o01,3,"[8.152, 8.02, 8.036, 8.208, 8.208, 8.128, 8.12...","[8.672, 8.684, 8.648, 8.632, 8.628, 8.672, 8.6...",14.913591,0.688938,1.089432,1.680000,2.391000,32.894737,62.955495,69.124424,261.545152,6.893892,5.0,5.0,1063.164772
4,f1o01,4,"[8.068, 8.004, 7.864, 7.9, 8.048, 7.996, 7.848...","[8.548, 8.524, 8.508, 8.528, 8.568, 8.552, 8.5...",16.444328,0.983089,0.837500,1.479200,2.752000,57.251908,62.337664,67.264574,14.326248,5.866205,5.0,5.0,1472.451584
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9469,f2y10,232,"[-0.10009765625, -0.09765625, -0.10009765625, ...","[0.0048828125, 0.0048828125, 0.0048828125, 0.0...",14.497019,0.197665,0.101241,1.556800,2.480000,52.631579,58.396608,66.079295,66.171494,6.985175,5.0,5.0,67.014924
9470,f2y10,233,"[-0.185546875, -0.1806640625, -0.18310546875, ...","[-0.126953125, -0.12939453125, -0.126953125, -...",12.471924,0.304567,0.516003,1.549600,3.345000,51.369863,58.250585,65.789474,56.823614,7.723637,5.0,5.0,176.674843
9471,f2y10,234,"[-0.01708984375, -0.01220703125, -0.01953125, ...","[0.54443359375, 0.5419921875, 0.5517578125, 0....",15.151441,0.219147,0.206771,1.567200,2.387000,49.180328,56.770476,62.761506,542.688370,6.556642,5.0,5.0,169.671251
9473,f2y10,236,"[0.1025390625, 0.07568359375, 0.0439453125, 0....","[-0.185546875, -0.185546875, -0.1806640625, -0...",14.860658,0.272106,0.304594,1.684667,2.696000,50.000000,56.347270,64.655172,63.884896,7.360403,6.0,6.0,211.406007


# Preprocessing

In [8]:
chunks_df = data_df #allias

In [9]:
chunks_df = chunks_df.dropna(subset=['Mean_Respiration_Rate','hrv'])

## Standardization

In [10]:
# Convert 'ecg' column to a 3D numpy array
X = np.array(chunks_df['ecg'].tolist())
Y = chunks_df[['Mean_Respiration_Rate','Total_Area_Trough_to_Peak','Total_Area_Peak_to_Trough',
            'Mean_latency_Trough_to_Peak','Mean_latency_Peak_to_Trough']].values

# Normalize each ECG chunk individually and handle NaNs
X_normalized = np.zeros_like(X)
for i in range(X.shape[0]):
    mean = np.mean(X[i])
    std = np.std(X[i])
    if std == 0:  # Avoid division by zero
        print('sid == 0')
        std = 1
    X_normalized[i] = (X[i] - mean) / std

In [11]:
X_feature = np.array(chunks_df[['hr_min','hr_mean','hr_max','hrv','snr']])


# Normalize each ECG chunk individually and handle NaNs
X_feature_normalized = np.zeros_like(X_feature)
for i in range(X_feature.shape[1]):
    mean = np.mean(X_feature[:,i])
    std = np.std(X_feature[:,i])
    if std == 0:  # Avoid division by zero
        print('sid == 0')
        std = 1
    X_feature_normalized[:,i] = (X_feature[:,i] - mean) / std

In [12]:
print(X_normalized.shape)
print(X_feature_normalized.shape)
print(Y.shape)

(8099, 7500)
(8099, 5)
(8099, 5)


In [13]:
# Remove Nan value
nan_rows = np.isnan(X_normalized).any(axis=1)

# Remove rows with NaNs
X_normalized = X_normalized[~nan_rows]
X_feature_normalized = X_feature_normalized[~nan_rows]
Y = Y[~nan_rows]
cleaned_chunks_df = chunks_df[~nan_rows]

print(nan_rows.sum())
print(f"NaN in X_normalized after cleaning: {np.isnan(X_normalized).sum()}, Inf in X_normalized: {np.isinf(X_normalized).sum()}")

0
NaN in X_normalized after cleaning: 0, Inf in X_normalized: 0


In [14]:
print(X_normalized.shape)
print(X_feature_normalized.shape)
print(Y.shape)

(8099, 7500)
(8099, 5)
(8099, 5)


In [15]:
# # Calculate the 2.5th and 97.5th percentiles
# for i in range(Y.shape[1]):
#     lower_percentile = np.percentile(Y[:,i], 0.5)
#     upper_percentile = np.percentile(Y[:,i], 99.5)



#     # Verify the clipping
#     print('\n', i)
#     print(f"1th percentile: {lower_percentile}")
#     print(f"99th percentile: {upper_percentile}")
#     print(f"Before clipping: min = {Y[:,i].min()}, max = {Y[:,i].max()}")
#     # Clip the values in y_train
#     Y[:,i] = np.clip(Y[:,i], lower_percentile, upper_percentile)
#     print(f"After clipping: min = {Y[:,i].min()}, max = {Y[:,i].max()}")

#     # Continue with training the model using y_train_clipped

Spliting

In [16]:
from sklearn.model_selection import train_test_split


subjects = cleaned_chunks_df['subject_id'].unique()
train_val_subjects, test_subjects = train_test_split(subjects, test_size=0.2, random_state=96)
train_subjects, val_subjects = train_test_split(train_val_subjects, test_size=0.15, random_state=96)

# Create boolean masks for train, validation, and test subjects
train_mask = cleaned_chunks_df['subject_id'].isin(train_subjects)
val_mask = cleaned_chunks_df['subject_id'].isin(val_subjects)
test_mask = cleaned_chunks_df['subject_id'].isin(test_subjects)

# Use these masks to create the actual train, validation, and test datasets
train_data = cleaned_chunks_df[train_mask]
val_data = cleaned_chunks_df[val_mask]
test_data = cleaned_chunks_df[test_mask]

# Now you have train, validation, and test datasets based on the subjects
test_subject_id = cleaned_chunks_df[test_mask]['subject_id'].to_numpy()

In [17]:
print(train_subjects)
print(val_subjects)
print(test_subjects)

['f1o06' 'f1y09' 'f1o03' 'f2o02' 'f2o05' 'f1o09' 'f1o08' 'f2y10' 'f1o10'
 'f2o04' 'f2y08' 'f1o05' 'f1o04' 'f1y01' 'f2y06' 'f2y07' 'f1o02' 'f2y04'
 'f2o09' 'f1y05' 'f2o07' 'f2o10' 'f2o01' 'f1y06' 'f2o03' 'f1o07' 'f2y09']
['f1y08' 'f1y04' 'f1y03' 'f2y02' 'f2y01']
['f1y10' 'f1o01' 'f1y07' 'f2o06' 'f2y03' 'f1y02' 'f2o08' 'f2y05']


In [18]:
X_normalized

array([[-0.00809941,  0.40926564, -0.34957991, ...,  0.27014396,
         0.40926564, -0.26104793],
       [ 0.26904106,  1.15557631,  1.43765571, ..., -0.33541479,
         0.17501459,  0.17501459],
       [-0.7231427 , -0.7231427 , -0.09334483, ..., -0.98677902,
        -0.64991039, -0.07869837],
       ...,
       [-0.02289313,  0.00324655, -0.03596296, ...,  2.7609822 ,
         1.85916343,  0.65673841],
       [ 0.90269696,  0.74909981,  0.56757591, ..., -0.27022672,
        -0.34004361, -0.34004361],
       [-0.79813056, -0.73697144, -0.67581233, ..., -0.46175542,
        -0.37001674, -0.33943718]], shape=(8099, 7500))

In [19]:
# Apply the masks to create train, validation, and test sets
X_train = np.expand_dims(X_normalized[train_mask], 2)
Y_train = Y[train_mask] 
X_val = np.expand_dims(X_normalized[val_mask], 2)
Y_val = Y[val_mask]
X_test = np.expand_dims(X_normalized[test_mask], 2)
Y_test = Y[test_mask]

# Print shapes of the datasets to verify
print(f"X_train shape: {X_train.shape}")
print(f"X_val shape: {X_val.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_train shape: {Y_train.shape}")
print(f"y_val shape: {Y_val.shape}")
print(f"y_test shape: {Y_test.shape}")
print(f"test_subject_id shape: {test_subject_id.shape}")

X_train shape: (5445, 7500, 1)
X_val shape: (1021, 7500, 1)
X_test shape: (1633, 7500, 1)
y_train shape: (5445, 5)
y_val shape: (1021, 5)
y_test shape: (1633, 5)
test_subject_id shape: (1633,)


In [20]:
# Apply the masks to create train, validation, and test sets
X_train_feature = X_feature_normalized[train_mask]
X_val_feature = X_feature_normalized[val_mask]
X_test_feature = X_feature_normalized[test_mask]


# Print shapes of the datasets to verify
print(f"X_train_feature shape: {X_train_feature.shape}")
print(f"X_val_feature shape: {X_val_feature.shape}")
print(f"X_test_feature shape: {X_test_feature.shape}")


X_train_feature shape: (5445, 5)
X_val_feature shape: (1021, 5)
X_test_feature shape: (1633, 5)


In [21]:
def compute_freq_domain(X, SAMPLE_RATE = 250, N_SAMPLES = 30*250):
    X_freq = []
    for i in range(X.shape[0]):
        data = X_train[i]
        frequency_domain = np.fft.fft(data)
        #frequencies = np.fft.fftfreq(N_SAMPLES, 1/SAMPLE_RATE)
        X_freq.append(np.abs(frequency_domain))
    X_freq = np.array(X_freq)
    return X_freq
    

In [22]:
X_train_freq = compute_freq_domain(X_train)
X_val_freq =  compute_freq_domain(X_val)
X_test_freq = compute_freq_domain(X_test)

print(f"X_train_freq shape: {X_train_freq.shape}")
print(f"X_val_freq shape: {X_val_freq.shape}")
print(f"X_test_freq shape: {X_test_freq.shape}")

X_train_freq shape: (5445, 7500, 1)
X_val_freq shape: (1021, 7500, 1)
X_test_freq shape: (1633, 7500, 1)


In [23]:
import pywt

def extract_wavelet_features(X, wavelet='db4', level=4):
    n_samples = X.shape[0]
    
    #calculate n_feature
    n_features = 0
    temp = pywt.wavedec(np.zeros(X.shape[1]), wavelet, level=level)
    for i in temp:
        n_features += len(i)

    X_features = np.zeros((n_samples, n_features))
    for i in range(n_samples):
        current_signal = X[i, :, 0]
        
        coeffs = pywt.wavedec(current_signal, wavelet, level=level)
        features = np.concatenate(coeffs)
        
        X_features[i, :] = features
    return X_features

In [24]:
X_train_wavelet = extract_wavelet_features(X_train, wavelet='db4', level=4)
X_val_wavelet =  extract_wavelet_features(X_val, wavelet='db4', level=4)
X_test_wavelet = extract_wavelet_features(X_test, wavelet='db4', level=4)

print(f"X_train_wavelet shape: {X_train_wavelet.shape}")
print(f"X_val_wavelet shape: {X_val_wavelet.shape}")
print(f"X_test_wavelet shape: {X_test_wavelet.shape}")

X_train_wavelet shape: (5445, 7526)
X_val_wavelet shape: (1021, 7526)
X_test_wavelet shape: (1633, 7526)


In [25]:
output_ranges = np.max(Y_train, axis=0) - np.min(Y_train, axis=0)

# Calculate the inverse of the range to use as scaling factors
scaling_factors = 1 / output_ranges

# Normalize scaling factors to sum up to 1
scaling_factors /= scaling_factors.sum()

print(scaling_factors) 

[0.05246714 0.15850716 0.12722465 0.41182092 0.24998013]


# Build Model

In [26]:
EPOCHS = 200
BATCH_SIZE = 64
PATIENCE = 20

In [27]:
cleaned_chunks_df[test_mask]['subject_id'].to_numpy()

array(['f1o01', 'f1o01', 'f1o01', ..., 'f2y05', 'f2y05', 'f2y05'],
      shape=(1633,), dtype=object)

In [28]:
#create model
def create_CNN_model_5_output(shape, start_neuron = 64, kernel_size=9, strides_size=1, max_pool_size=3, dropout=0.3, padding='valid', delta=1.0):    

    input_layer = tf.keras.Input((shape[1], shape[2])) 

    
    conv_1 = tf.keras.layers.Conv1D(start_neuron * 1, kernel_size, strides=strides_size,  padding=padding, activation=tf.keras.layers.LeakyReLU(),kernel_regularizer=tf.keras.regularizers.l2(0.01))(input_layer)
    conv_1 = tf.keras.layers.MaxPool1D(max_pool_size,  padding=padding)(conv_1)
    conv_1 = tf.keras.layers.Dropout(dropout)(conv_1)
    
    conv_2 = tf.keras.layers.Conv1D(start_neuron * 2, kernel_size, strides=strides_size, padding=padding, activation=tf.keras.layers.LeakyReLU(),kernel_regularizer=tf.keras.regularizers.l2(0.01))(conv_1)
    conv_2 = tf.keras.layers.MaxPool1D(max_pool_size,  padding=padding)(conv_2)
    conv_2 = tf.keras.layers.Dropout(dropout)(conv_2)    
    
    conv_3 = tf.keras.layers.Conv1D(start_neuron * 4, kernel_size, strides=strides_size, padding=padding, activation=tf.keras.layers.LeakyReLU(),kernel_regularizer=tf.keras.regularizers.l2(0.01))(conv_2)
    conv_3 = tf.keras.layers.MaxPool1D(max_pool_size,  padding=padding)(conv_3)
    conv_3 = tf.keras.layers.Dropout(dropout)(conv_3)
    
    
    flattern_layer = tf.keras.layers.Flatten()(conv_3)
    flattern_layer = tf.keras.layers.Dense(start_neuron * 2,activation=tf.keras.layers.LeakyReLU())(flattern_layer)
    
#     output_0 = tf.keras.layers.Dense(32,activation='relu',)(flattern_layer)
#     output_0 = tf.keras.layers.Dropout(dropout)(output_0)    
    output_0 = tf.keras.layers.Dense(1,activation='linear',name='mean_rr')(flattern_layer)
    
    
#     output_1 = tf.keras.layers.Dense(32,activation='relu',)(flattern_layer)
#     output_1 = tf.keras.layers.Dropout(dropout)(output_1)    
    output_1 = tf.keras.layers.Dense(1,activation='linear',name='area_t_to_p')(flattern_layer)
    
#     output_2 = tf.keras.layers.Dense(32,activation='relu',)(flattern_layer)
#     output_2 = tf.keras.layers.Dropout(dropout)(output_2)    
    output_2 = tf.keras.layers.Dense(1,activation='linear',name='area_p_to_t')(flattern_layer)
    
#     output_3 = tf.keras.layers.Dense(32,activation='relu',)(flattern_layer)
#     output_3 = tf.keras.layers.Dropout(dropout)(output_3)   
    output_3 = tf.keras.layers.Dense(1,activation='linear',name='time_t_to_p')(flattern_layer)
    
#     output_4 = tf.keras.layers.Dense(32,activation='relu',)(flattern_layer)    
#     output_4 = tf.keras.layers.Dropout(dropout)(output_4)   
    output_4 = tf.keras.layers.Dense(1,activation='linear',name='time_p_to_t')(flattern_layer)

    
    model = tf.keras.Model(input_layer, [output_0,output_1,output_2,output_3,output_4])
    loss_weights = [scaling_factors[0]*5,scaling_factors[1],scaling_factors[2],scaling_factors[3],scaling_factors[4]]   
    model.compile(optimizer = 'adam',loss = [tf.keras.losses.Huber(delta=delta),tf.keras.losses.Huber(delta=delta),tf.keras.losses.Huber(delta=delta),tf.keras.losses.Huber(delta=delta),tf.keras.losses.Huber(delta=delta)],loss_weights=loss_weights)
    return model

In [29]:
#create model
def create_CNN_LSTM_model_5_output(shape, start_neuron = 64, kernel_size=9, strides_size=1, max_pool_size=3, dropout=0.3, padding='valid', delta=1.0):    
    input_layer = tf.keras.Input((shape[1], shape[2])) 
    
    conv_1 = tf.keras.layers.Conv1D(start_neuron * 1, kernel_size, strides=strides_size,  padding=padding, activation=tf.keras.layers.LeakyReLU(),kernel_regularizer=tf.keras.regularizers.l2(0.01))(input_layer)
    conv_1 = tf.keras.layers.MaxPool1D(max_pool_size,  padding=padding)(conv_1)
    conv_1 = tf.keras.layers.Dropout(dropout)(conv_1)
    
    conv_2 = tf.keras.layers.Conv1D(start_neuron * 2, kernel_size, strides=strides_size, padding=padding, activation=tf.keras.layers.LeakyReLU(),kernel_regularizer=tf.keras.regularizers.l2(0.01))(conv_1)
    conv_2 = tf.keras.layers.MaxPool1D(max_pool_size,  padding=padding)(conv_2)
    conv_2 = tf.keras.layers.Dropout(dropout)(conv_2)    
    
    lstm_1 = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(start_neuron, return_sequences=True))(conv_2)
    lstm_1 = tf.keras.layers.Dropout(dropout)(lstm_1)

    lstm_2 = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(start_neuron * 2, return_sequences=True))(lstm_1)
    lstm_2 = tf.keras.layers.Dropout(dropout)(lstm_2)
    lstm_2 = tf.keras.layers.Flatten()(lstm_2)
    
    output_layer = tf.keras.layers.Dense(start_neuron*2,activation=tf.keras.layers.LeakyReLU())(lstm_2)
    output_layer = tf.keras.layers.Dropout(dropout)(output_layer)
    
    #output_0 = tf.keras.layers.Dense(start_neuron*2,activation='relu')(output_layer)
    output_0 = tf.keras.layers.Dense(1,activation='linear',name='mean_rr')(output_layer)

    #output_1 = tf.keras.layers.Dense(start_neuron*2,activation='relu')(output_layer)
    output_1 = tf.keras.layers.Dense(1,activation='linear',name='area_t_to_p')(output_layer)

    #output_2 = tf.keras.layers.Dense(start_neuron*2,activation='relu')(output_layer)
    output_2 = tf.keras.layers.Dense(1,activation='linear',name='area_p_to_t')(output_layer)

    #output_3 = tf.keras.layers.Dense(start_neuron*2,activation='relu')(output_layer)
    output_3 = tf.keras.layers.Dense(1,activation='linear',name='time_t_to_p')(output_layer)

    #output_4 = tf.keras.layers.Dense(start_neuron*2,activation='relu')(output_layer)
    output_4 = tf.keras.layers.Dense(1,activation='linear',name='time_p_to_t')(output_layer)

    
    model = tf.keras.Model(input_layer, [output_0,output_1,output_2,output_3,output_4])
    loss_weights = [scaling_factors[0]*5,scaling_factors[1],scaling_factors[2],scaling_factors[3],scaling_factors[4]]   
    #loss_weights = [scaling_factors[0]*5,0,0,0,0]  
    model.compile(optimizer = 'adam',loss = [tf.keras.losses.Huber(delta=delta),tf.keras.losses.Huber(delta=delta),tf.keras.losses.Huber(delta=delta),tf.keras.losses.Huber(delta=delta),tf.keras.losses.Huber(delta=delta)],loss_weights=loss_weights)
    return model

In [30]:
#create model
def create_time_wavelet_model_5_output(shape, start_neuron = 64, kernel_size=9, strides_size=1, max_pool_size=3, dropout=0.3, padding='valid', delta=1.0):    
    
    input_layer = tf.keras.Input((shape[1], shape[2]), name='input_time') 
    input_layer_wavelet = tf.keras.Input((shape[4], shape[2]), name='input_wavelet') 

    #time domain
    conv_1 = tf.keras.layers.Conv1D(start_neuron * 1, kernel_size, strides=strides_size,  padding=padding, activation=tf.keras.layers.LeakyReLU(),kernel_regularizer=tf.keras.regularizers.l2(0.01))(input_layer)
    conv_1 = tf.keras.layers.Conv1D(start_neuron * 1, kernel_size, strides=strides_size,  padding=padding, activation=tf.keras.layers.LeakyReLU(),kernel_regularizer=tf.keras.regularizers.l2(0.01))(conv_1)
    conv_1 = tf.keras.layers.MaxPool1D(max_pool_size,  padding=padding)(conv_1)
    conv_1 = tf.keras.layers.Dropout(dropout)(conv_1)
    
    conv_2 = tf.keras.layers.Conv1D(start_neuron * 2, kernel_size, strides=strides_size, padding=padding, activation=tf.keras.layers.LeakyReLU(),kernel_regularizer=tf.keras.regularizers.l2(0.01))(conv_1)
    conv_2 = tf.keras.layers.Conv1D(start_neuron * 2, kernel_size, strides=strides_size, padding=padding, activation=tf.keras.layers.LeakyReLU(),kernel_regularizer=tf.keras.regularizers.l2(0.01))(conv_2)
    conv_2 = tf.keras.layers.MaxPool1D(max_pool_size,  padding=padding)(conv_2)
    conv_2 = tf.keras.layers.Dropout(dropout)(conv_2)    
    
    lstm_1 = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(start_neuron, return_sequences=True))(conv_2)
    lstm_1 = tf.keras.layers.Dropout(dropout)(lstm_1)

    lstm_2 = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(start_neuron * 2, return_sequences=True))(lstm_1)
    lstm_2 = tf.keras.layers.Dropout(dropout)(lstm_2)
    lstm_2 = tf.keras.layers.Flatten()(lstm_2)

    #wavelet domain
    conv_1_wavelet = tf.keras.layers.Conv1D(start_neuron * 1, kernel_size, strides=strides_size,  padding=padding, activation=tf.keras.layers.LeakyReLU(),kernel_regularizer=tf.keras.regularizers.l2(0.01))(input_layer_wavelet)
    conv_1_wavelet = tf.keras.layers.MaxPool1D(max_pool_size,  padding=padding)(conv_1_wavelet)
    conv_1_wavelet = tf.keras.layers.Dropout(dropout)(conv_1_wavelet)
    
    conv_2_wavelet = tf.keras.layers.Conv1D(start_neuron * 2, kernel_size, strides=strides_size, padding=padding, activation=tf.keras.layers.LeakyReLU(),kernel_regularizer=tf.keras.regularizers.l2(0.01))(conv_1_wavelet)
    conv_2_wavelet = tf.keras.layers.MaxPool1D(max_pool_size,  padding=padding)(conv_2_wavelet)
    conv_2_wavelet = tf.keras.layers.Dropout(dropout)(conv_2_wavelet)    
    
    lstm_1_wavelet = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(start_neuron, return_sequences=True))(conv_2_wavelet)
    lstm_1_wavelet = tf.keras.layers.Dropout(dropout)(lstm_1_wavelet)

    lstm_2_wavelet = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(start_neuron * 2, return_sequences=True))(lstm_1_wavelet)
    lstm_2_wavelet = tf.keras.layers.Dropout(dropout)(lstm_2_wavelet)
    lstm_2_wavelet = tf.keras.layers.Flatten()(lstm_2_wavelet)    

    concat_layer = tf.keras.layers.Concatenate()([lstm_2, lstm_2_wavelet])
    
    output_layer = tf.keras.layers.Dense(start_neuron*2,activation=tf.keras.layers.LeakyReLU())(concat_layer)
    output_layer = tf.keras.layers.Dropout(dropout)(output_layer)
    
    #output_0 = tf.keras.layers.Dense(start_neuron*2,activation='relu')(output_layer)
    output_0 = tf.keras.layers.Dense(1,activation='linear',name='mean_rr')(output_layer)

    #output_1 = tf.keras.layers.Dense(start_neuron*2,activation='relu')(output_layer)
    output_1 = tf.keras.layers.Dense(1,activation='linear',name='area_t_to_p')(output_layer)

    #output_2 = tf.keras.layers.Dense(start_neuron*2,activation='relu')(output_layer)
    output_2 = tf.keras.layers.Dense(1,activation='linear',name='area_p_to_t')(output_layer)

    #output_3 = tf.keras.layers.Dense(start_neuron*2,activation='relu')(output_layer)
    output_3 = tf.keras.layers.Dense(1,activation='linear',name='time_t_to_p')(output_layer)

    #output_4 = tf.keras.layers.Dense(start_neuron*2,activation='relu')(output_layer)
    output_4 = tf.keras.layers.Dense(1,activation='linear',name='time_p_to_t')(output_layer)

    
    model = tf.keras.Model(inputs=[input_layer, input_layer_wavelet], outputs=[output_0,output_1,output_2,output_3,output_4])
    loss_weights = [scaling_factors[0]*5,scaling_factors[1],scaling_factors[2],scaling_factors[3],scaling_factors[4]]   
    #loss_weights = [scaling_factors[0]*5,0,0,0,0]  
    model.compile(optimizer = 'adam',loss = [tf.keras.losses.Huber(delta=delta),tf.keras.losses.Huber(delta=delta),tf.keras.losses.Huber(delta=delta),tf.keras.losses.Huber(delta=delta),tf.keras.losses.Huber(delta=delta)],loss_weights=loss_weights)
    #model.compile(optimizer = 'adam',loss = ['mae','mae','mae','mae','mae'],loss_weights=loss_weights)
    return model

In [31]:
#create model
def custom_cnn_layers(input_layer, start_neuron, kernel_size, strides_size, max_pool_size, dropout, padding='valid', delta=1.0):
    layer = tf.keras.layers.Conv1D(start_neuron, kernel_size, strides=strides_size,  padding=padding, activation=tf.keras.layers.LeakyReLU(),kernel_regularizer=tf.keras.regularizers.l2(0.01))(input_layer)
    layer = tf.keras.layers.Conv1D(start_neuron, kernel_size, strides=strides_size,  padding=padding, activation=tf.keras.layers.LeakyReLU(),kernel_regularizer=tf.keras.regularizers.l2(0.01))(layer)
    layer = tf.keras.layers.MaxPool1D(max_pool_size,  padding=padding)(layer)
    layer = tf.keras.layers.Dropout(dropout)(layer)
    return layer

    
def create_3_input_model_5_output(shape, start_neuron = 64, kernel_size=9, strides_size=1, max_pool_size=3, dropout=0.3, padding='valid', delta=1.0):
    input_layer = tf.keras.Input((shape[1], shape[2]), name='input_time') 
    input_layer_wavelet = tf.keras.Input((shape[4], shape[2]), name='input_wavelet') 
    input_layer_feature = tf.keras.Input((shape[6],),name='input_feature')
    
    #time domain
    conv_1 = custom_cnn_layers(input_layer, start_neuron = start_neuron, kernel_size = kernel_size, strides_size = strides_size, max_pool_size = 5, dropout = dropout, padding='same', delta=1.0)
    conv_2 = custom_cnn_layers(conv_1, start_neuron = start_neuron * 2, kernel_size = kernel_size, strides_size = strides_size, max_pool_size = 5, dropout = dropout, padding='same', delta=1.0)
    
    conv_3 = custom_cnn_layers(conv_2, start_neuron = start_neuron * 4, kernel_size = kernel_size, strides_size = strides_size, max_pool_size = 3, dropout = dropout, padding='same', delta=1.0)
    conv_4 = custom_cnn_layers(conv_3, start_neuron = start_neuron * 8, kernel_size = kernel_size, strides_size = strides_size, max_pool_size = 2, dropout = 0, padding='same', delta=1.0)
    conv_4 = tf.keras.layers.GlobalAveragePooling1D()(conv_4)
    conv_4 = tf.keras.layers.Dropout(dropout)(conv_4)

    lstm_1 = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(start_neuron * 2, return_sequences=True))(conv_2)
    #lstm_1 = tf.keras.layers.MaxPool1D(3,  padding=padding)(lstm_1)
    lstm_1 = tf.keras.layers.Dropout(dropout)(lstm_1)
    
    lstm_2 = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(start_neuron * 4, return_sequences=True))(lstm_1)
    lstm_2 = tf.keras.layers.GlobalAveragePooling1D()(lstm_2)
    lstm_2 = tf.keras.layers.Dropout(dropout)(lstm_2)

    #wavelet domain
    conv_1_wavelet = custom_cnn_layers(input_layer_wavelet, start_neuron = start_neuron, kernel_size = kernel_size, strides_size = strides_size, max_pool_size = 5, dropout = dropout, padding='same', delta=1.0)
    conv_2_wavelet = custom_cnn_layers(conv_1_wavelet, start_neuron = start_neuron * 2, kernel_size = kernel_size, strides_size = strides_size, max_pool_size = 5, dropout = dropout, padding='same', delta=1.0)
    
    conv_3_wavelet = custom_cnn_layers(conv_2_wavelet, start_neuron = start_neuron * 4, kernel_size = kernel_size, strides_size = strides_size, max_pool_size = 3, dropout = dropout, padding='same', delta=1.0)
    conv_4_wavelet = custom_cnn_layers(conv_3_wavelet, start_neuron = start_neuron * 8, kernel_size = kernel_size, strides_size = strides_size, max_pool_size = 2, dropout = 0, padding='same', delta=1.0)
    conv_4_wavelet = tf.keras.layers.GlobalAveragePooling1D()(conv_4_wavelet)
    conv_4_wavelet = tf.keras.layers.Dropout(dropout)(conv_4_wavelet)

    lstm_1_wavelet = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(start_neuron * 2, return_sequences=True))(conv_2_wavelet)
    #lstm_1_wavelet = tf.keras.layers.MaxPool1D(3,  padding=padding)(lstm_1_wavelet)
    lstm_1_wavelet = tf.keras.layers.Dropout(dropout)(lstm_1_wavelet)
    
    lstm_2_wavelet = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(start_neuron * 4, return_sequences=True))(lstm_1_wavelet)
    lstm_2_wavelet = tf.keras.layers.GlobalAveragePooling1D()(lstm_2_wavelet)
    lstm_2_wavelet = tf.keras.layers.Dropout(dropout)(lstm_2_wavelet)

    
    concat_layer = tf.keras.layers.Concatenate()([conv_4,lstm_2,conv_4_wavelet,lstm_2_wavelet,input_layer_feature])
    output_layer = tf.keras.layers.Dense(start_neuron*8,activation=tf.keras.layers.LeakyReLU())(concat_layer)
    output_layer = tf.keras.layers.Dropout(dropout)(output_layer)
    
    #output_0 = tf.keras.layers.Dense(start_neuron*2,activation='relu')(output_layer)
    output_0 = tf.keras.layers.Dense(1,activation='linear',name='mean_rr')(output_layer)

    #output_1 = tf.keras.layers.Dense(start_neuron*2,activation='relu')(output_layer)
    output_1 = tf.keras.layers.Dense(1,activation='linear',name='area_t_to_p')(output_layer)

    #output_2 = tf.keras.layers.Dense(start_neuron*2,activation='relu')(output_layer)
    output_2 = tf.keras.layers.Dense(1,activation='linear',name='area_p_to_t')(output_layer)

    #output_3 = tf.keras.layers.Dense(start_neuron*2,activation='relu')(output_layer)
    output_3 = tf.keras.layers.Dense(1,activation='linear',name='time_t_to_p')(output_layer)

    #output_4 = tf.keras.layers.Dense(start_neuron*2,activation='relu')(output_layer)
    output_4 = tf.keras.layers.Dense(1,activation='linear',name='time_p_to_t')(output_layer)

    
    model = tf.keras.Model(inputs=[input_layer, input_layer_wavelet, input_layer_feature], outputs=[output_0,output_1,output_2,output_3,output_4])
    loss_weights = [scaling_factors[0]*5,scaling_factors[1],scaling_factors[2],scaling_factors[3],scaling_factors[4]]   
    #loss_weights = [scaling_factors[0]*5,0,0,0,0]  
    model.compile(optimizer = 'adam',loss = [tf.keras.losses.Huber(delta=delta),tf.keras.losses.Huber(delta=delta),tf.keras.losses.Huber(delta=delta),tf.keras.losses.Huber(delta=delta),tf.keras.losses.Huber(delta=delta)],loss_weights=loss_weights)
    #model.compile(optimizer = 'adam',loss = ['mae','mae','mae','mae','mae'],loss_weights=loss_weights)
    return model

In [32]:
reduce_LR_On_Plateau = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss',factor=0.5,patience=PATIENCE//2,verbose=1, min_delta=0.00001,)
early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=PATIENCE, restore_best_weights = True, mode = 'min')

In [33]:
model = create_3_input_model_5_output((X_train.shape + X_train_wavelet.shape + X_train_feature.shape), start_neuron = 32, kernel_size= 11, strides_size=1, 
                                  max_pool_size= 5, dropout=0.3,padding='same', delta=0.75)
model.summary()

history = model.fit([X_train,X_train_wavelet,X_train_feature], [Y_train[:,0],Y_train[:,1],Y_train[:,2],Y_train[:,3],Y_train[:,4]], 
                    validation_data=([X_val,X_val_wavelet,X_val_feature], [Y_val[:,0],Y_val[:,1],Y_val[:,2],Y_val[:,3],Y_val[:,4]]), 
                    epochs=EPOCHS, batch_size=BATCH_SIZE,verbose=1, shuffle=True, callbacks=[reduce_LR_On_Plateau,early_stopping])

W0000 00:00:1760991182.250356   11042 gpu_device.cc:2430] TensorFlow was not built with CUDA kernel binaries compatible with compute capability 12.0. CUDA kernels will be jit-compiled from PTX, which could take 30 minutes or longer.
I0000 00:00:1760991182.250517   11042 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13121 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 5070 Ti, pci bus id: 0000:01:00.0, compute capability: 12.0
2025-10-20 16:13:02.895260: W tensorflow/compiler/mlir/tools/kernel_gen/tf_gpu_runtime_wrappers.cc:40] 'cuModuleLoadData(&module, data)' failed with 'CUDA_ERROR_INVALID_PTX'

2025-10-20 16:13:02.895311: W tensorflow/compiler/mlir/tools/kernel_gen/tf_gpu_runtime_wrappers.cc:40] 'cuModuleGetFunction(&function, module, kernel_name)' failed with 'CUDA_ERROR_INVALID_HANDLE'

2025-10-20 16:13:02.895324: W tensorflow/core/framework/op_kernel.cc:1844] INTERNAL: 'cuLaunchKernel(function, gridX, gridY, gridZ, blockX, blockY, blockZ

InternalError: {{function_node __wrapped__Cast_device_/job:localhost/replica:0/task:0/device:GPU:0}} 'cuLaunchKernel(function, gridX, gridY, gridZ, blockX, blockY, blockZ, 0, reinterpret_cast<CUstream>(stream), params, nullptr)' failed with 'CUDA_ERROR_INVALID_HANDLE' [Op:Cast] name: 

# Evaluation

In [ ]:
Y_pred = model.predict([X_test,X_test_wavelet,X_test_feature],verbose = 1)
Y_pred

In [ ]:
# Plot training & validation loss values
plt.figure(figsize=(12, 6))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(loc='upper right')
plt.show()

# # Plot training & validation MAE values
# plt.figure(figsize=(12, 6))
# plt.plot(history.history['mae'], label='Training MAE')
# plt.plot(history.history['val_mae'], label='Validation MAE')
# plt.title('Model MAE')
# plt.ylabel('MAE')
# plt.xlabel('Epoch')
# plt.legend(loc='upper right')
# plt.show()


In [ ]:
Y_test

In [ ]:
Y_pred

In [ ]:
# Function to fit and plot piecewise linear regression
def piecewise_regression(x, y, breakpoints, ax):
    segments = np.split(np.argsort(x), np.searchsorted(x[np.argsort(x)], breakpoints))
    segment_texts = []
    
    for i,segment in enumerate(segments):
        if len(segment) > 1:  # Fit line if segment contains more than one point
            slope, intercept, _, _, _ = linregress(x[segment], y[segment])
            ax.plot(x[segment], slope * x[segment] + intercept, color='red', linewidth=2)
            
            # Calculate Pearson correlation coefficient and p-value for each segment
            corr, p_val = pearsonr(x[segment], y[segment])
            spearman_corr, spearman_p_val = spearmanr(x[segment], y[segment])
            segment_texts.append(f'Segment {i}: Pearsonr: {corr:.2f},  Spearman: {spearman_corr:.2f}')
    
    return segment_texts
# Get a list of unique subject IDs and create a color map
unique_subject_ids = np.unique(test_subject_id)
colors = plt.colormaps['tab20'](np.linspace(0, 1, len(unique_subject_ids)))
color_map = {subject_id: colors[i] for i, subject_id in enumerate(unique_subject_ids)}

# Initialize the figure and subplots
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.flatten()
output_names = ['Mean Respiration Rate', 'Area T to P', 'Area P to T', 'Time T to P', 'Time P to T']

for i in range(5):
    # Calculate the Pearson correlation coefficient
    correlation, p_value = pearsonr(Y_test[:, i], Y_pred[i].flatten())
    spearman_corr, spearman_p_value = spearmanr(Y_test[:, i], Y_pred[i].flatten())
    print(f"Output {i}: Pearson Correlation: {correlation}")
    print(f"Output {i}: P-value: {p_value}")

    # # Create scatter plot
    # axes[i].scatter(Y_test[:, i], Y_pred[i].flatten(), alpha=0.6, edgecolors='w', linewidth=0.5, label='Data points')
    
    for subject_id in unique_subject_ids:
        mask = test_subject_id == subject_id
        axes[i].scatter(Y_test[mask, i], Y_pred[i].flatten()[mask], alpha=0.6, edgecolors='w', linewidth=0.5, label=f'Subject {subject_id}', color=color_map[subject_id])
    # Define breakpoints
    breakpoints = np.percentile(Y_test[:, i], [33, 66])

    # Fit and plot piecewise regression
    segment_texts = piecewise_regression(Y_test[:, i], Y_pred[i].flatten(), breakpoints, axes[i])

    axes[i].set_title(f'Actual vs Predicted {output_names[i]}')
    axes[i].set_xlabel('Actual')
    axes[i].set_ylabel('Predicted')
    axes[i].grid(True)
    axes[i].legend(loc='upper right')

    # Add text box with correlation and p-value
    textstr = f'Pearson Correlation: {correlation:.2f}, Spearman: {spearman_corr:.2f}\n' + '\n'.join(segment_texts)
    props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
    axes[i].text(0.05, 0.95, textstr, transform=axes[i].transAxes, fontsize=10,
                 verticalalignment='top', bbox=props)

# Plot the model loss
axes[5].plot(history.history['loss'], label='Training Loss')
axes[5].plot(history.history['val_loss'], label='Validation Loss')
axes[5].set_title('Model Loss')
axes[5].set_ylabel('Loss')
axes[5].set_xlabel('Epoch')
axes[5].legend()

# Adjust layout to prevent overlap
plt.tight_layout()

# Ensure x and y axes have the same scale
for i, ax in enumerate(axes):
    if i < len(output_names):
        ax.set_xlim([min(Y_test[:, i].min(), Y_pred[i].min()), max(Y_test[:, i].max(), Y_pred[i].max())])
        ax.set_ylim([min(Y_test[:, i].min(), Y_pred[i].min()), max(Y_test[:, i].max(), Y_pred[i].max())])

plt.show()

In [ ]:
np.unique(test_subject_id)

In [ ]:
# list_id = ['f2y08']
# list_id = np.unique(test_subject_id)
# num_examples_to_plot = 10

# for current_subject_id in np.unique(test_subject_id):
    
#     # Shuffle and select a few examples from the test set
#     mask = (test_subject_id == current_subject_id)
#     Y_pred_filtered_rr = Y_pred[0][mask].flatten() 
#     Y_pred_filtered_area_T_to_P = Y_pred[1][mask].flatten()
#     Y_pred_filtered_area_P_to_T = Y_pred[2][mask].flatten()
#     Y_pred_filtered_time_T_to_P = Y_pred[3][mask].flatten()
#     Y_pred_filtered_time_P_to_T = Y_pred[4][mask].flatten()
    
#     Y_true_filtered_rr = Y_test[mask][:,0]
#     Y_true_filtered_area_T_to_P = Y_test[mask][:,1]
#     Y_true_filtered_area_P_to_T = Y_test[mask][:,2]
#     Y_true_filtered_time_T_to_P = Y_test[mask][:,3]
#     Y_true_filtered_time_P_to_T = Y_test[mask][:,4]
    
#     X_test_filtered = X_test[mask]
    
    
    
#     #Plot the selected examples from the test set
#     fig, axes = plt.subplots(num_examples_to_plot, 1, figsize=(12, 3*num_examples_to_plot))
#     for i, ax in enumerate(axes):
#         idx = random.randint(0, len(X_test_filtered)-1)
#         print(idx)
#         # Plot the sample
#         ax.plot(X_test_filtered[idx].flatten())
#         ax.set_title(f'Subject ID: {current_subject_id}. Test Sample: {idx}')
    
#         #Adding text box for comparison
#         textstr = '\n'.join((
#             f'             [Respiration Rate, Area Trough to Peak, Area Peak to Trough, Time Trough to Peak, Time Peak to Trough]', 
#             f'Predict: [{Y_pred_filtered_rr[idx]:.2f},  {Y_pred_filtered_area_T_to_P[idx]:22.2f},  {Y_pred_filtered_area_P_to_T[idx]:27.2f},  {Y_pred_filtered_time_T_to_P[idx]:27.2f},  {Y_pred_filtered_time_P_to_T[idx]:27.2f}]',
#             f'True:     [{Y_true_filtered_rr[idx]:.2f},  {Y_true_filtered_area_T_to_P[idx]:22.2f},  {Y_true_filtered_area_P_to_T[idx]:27.2f},  {Y_true_filtered_time_T_to_P[idx]:27.2f},  {Y_true_filtered_time_P_to_T[idx]:27.2f}]',
#         ))
#         ax.text(0.05, 0.95, textstr, transform=ax.transAxes, fontsize=10,
#                 verticalalignment='top', bbox=dict(facecolor='yellow', alpha=0.8))
    
#     plt.tight_layout()
    
#     SAVE_DIR = OUTPUT_DIR + 'sample/'
#     if(not(os.path.exists(SAVE_DIR))):
#         os.makedirs(SAVE_DIR)
#     plt.savefig(SAVE_DIR + f'sample_{current_subject_id}.png')
#     #plt.show()
#     plt.close()

# Grid Search

In [ ]:
# import numpy as np
# import tensorflow as tf
# from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
# from scipy.stats import pearsonr, linregress
# import matplotlib.pyplot as plt

# def piecewise_regression(x, y, breakpoints, ax):
#     segments = np.split(np.argsort(x), np.searchsorted(x[np.argsort(x)], breakpoints))
#     segment_texts = []
    
#     for i,segment in enumerate(segments):
#         if len(segment) > 1:  # Fit line if segment contains more than one point
#             slope, intercept, _, _, _ = linregress(x[segment], y[segment])
#             ax.plot(x[segment], slope * x[segment] + intercept, color='red', linewidth=2)
            
#             # Calculate Pearson correlation coefficient and p-value for each segment
#             corr, p_val = pearsonr(x[segment], y[segment])
#             spearman_corr, spearman_p_val = spearmanr(x[segment], y[segment])
#             segment_texts.append(f'Segment {i}: Pearsonr: {corr:.2f},  Spearman: {spearman_corr:.2f}')
    
#     return segment_texts
    
# if(not(os.path.exists(OUTPUT_DIR))):
#     os.makedirs(OUTPUT_DIR)
    
# # Define the grid search parameters
# param_grid = {
#     'start_neuron': [32],
#     'kernel_size': [9],
#     'strides_size': [3],
#     'max_pool_size': [5],
#     'dropout': [0.3],
#     'padding' :['same'],
#     'batch_size' : [64],
#     'huber_delta' : [0.75],
# }

# # Prepare callbacks


# # Number of epochs
# EPOCHS = 200
# PATIENCE = 20

# # Initialize a list to store the loss history for all configurations
# loss_history = []

# # Perform grid search
# for start_neuron in param_grid['start_neuron']:
#     for kernel_size in param_grid['kernel_size']:
#         for strides_size in param_grid['strides_size']:
#             for max_pool_size in param_grid['max_pool_size']:
#                 for dropout in param_grid['dropout']:
#                     for padding in param_grid['padding']:
#                         for batch_size in param_grid['batch_size']:
#                             for delta in param_grid['huber_delta']:
#                                 print(f'start_neuron={start_neuron}, kernel_size={kernel_size}, strides_size={strides_size}, max_pool_size={max_pool_size}, dropout={dropout}, padding={padding}, batch_size={batch_size}, huber_delta={delta}')
                                
#                                 reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss',factor=0.5,patience=PATIENCE//2,verbose=1, min_delta=0.00001,)
#                                 early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=PATIENCE, restore_best_weights = True, mode = 'min')
#                                 # Create the model
#                                 model = create_3_input_model_5_output((X_train.shape + X_train_wavelet.shape + X_train_feature.shape), start_neuron=start_neuron, kernel_size=kernel_size,
#                                                                   strides_size=strides_size, max_pool_size=max_pool_size, 
#                                                                   dropout=dropout, padding=padding, delta=delta)
#                                 #model.summary()
    
    
#                                 # Train the model
#                                 history = model.fit([X_train,X_train_wavelet,X_train_feature], [Y_train[:,0],Y_train[:,1],Y_train[:,2],Y_train[:,3],Y_train[:,4]], 
#                                                     validation_data=([X_val,X_val_wavelet,X_val_feature], [Y_val[:,0],Y_val[:,1],Y_val[:,2],Y_val[:,3],Y_val[:,4]]), 
#                                                     epochs=EPOCHS, batch_size=BATCH_SIZE,verbose=1, shuffle=True, 
#                                                     callbacks=[early_stopping, reduce_lr])



#                                 min_train_loss = min(history.history['loss'])
#                                 min_val_lost = min(history.history['val_loss'])
#                                 # Save loss history for the current configuration
#                                 config_loss_history = {
#                                     'config': f'start_neuron={start_neuron}, kernel_size={kernel_size}, strides_size={strides_size}, max_pool_size={max_pool_size}, dropout={dropout}, padding={padding}, batch_size={batch_size}, huber_delta={delta}',
#                                     'training_loss': history.history['loss'],
#                                     'validation_loss': history.history['val_loss'],
#                                     'min' : f'Best_train_loss{min_train_loss}, best_val_loss{min_val_lost}'
#                                 }
#                                 loss_history.append(config_loss_history)
    
#                                 # Evaluate the model
#                                 Y_pred = model.predict([X_test,X_test_wavelet,X_test_feature],verbose=0)

#                                 plt.clf()
#                                 unique_subject_ids = np.unique(test_subject_id)
#                                 colors = plt.colormaps['tab20'](np.linspace(0, 1, len(unique_subject_ids)))
#                                 color_map = {subject_id: colors[i] for i, subject_id in enumerate(unique_subject_ids)}


#                                 # Initialize the figure and subplots
#                                 # Create plots
#                                 fig, axes = plt.subplots(2, 3, figsize=(18, 12))
#                                 axes = axes.flatten()
#                                 output_names = ['Mean Respiration Rate', 'Area T to P', 'Area P to T', 'Time T to P', 'Time P to T']
    
#                                 for i in range(5):
#                                     # Calculate the Pearson correlation coefficient
#                                     correlation, p_value = pearsonr(Y_test[:, i], Y_pred[i].flatten())
#                                     print(f"Output {i}: Pearson Correlation: {correlation}")
#                                     print(f"Output {i}: P-value: {p_value}")
                                
#                                     # # Create scatter plot
#                                     # axes[i].scatter(Y_test[:, i], Y_pred[i].flatten(), alpha=0.6, edgecolors='w', linewidth=0.5, label='Data points')
                                    
#                                     for subject_id in unique_subject_ids:
#                                         mask = test_subject_id == subject_id
#                                         axes[i].scatter(Y_test[mask, i], Y_pred[i].flatten()[mask], alpha=0.6, edgecolors='w', linewidth=0.5, label=f'Subject {subject_id}', color=color_map[subject_id])
                                
#                                     # Define breakpoints
#                                     breakpoints = np.percentile(Y_test[:, i], [33, 66])
                                
#                                     # Fit and plot piecewise regression
#                                     segment_texts = piecewise_regression(Y_test[:, i], Y_pred[i].flatten(), breakpoints, axes[i])
                                
#                                     axes[i].set_title(f'Actual vs Predicted {output_names[i]}')
#                                     axes[i].set_xlabel('Actual')
#                                     axes[i].set_ylabel('Predicted')
#                                     axes[i].grid(True)
#                                     axes[i].legend()
                                
#                                     # Add text box with correlation and p-value
#                                     textstr = f'Pearson Correlation: {correlation:.2f}\n' + '\n'.join(segment_texts)
#                                     props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
#                                     axes[i].text(0.05, 0.95, textstr, transform=axes[i].transAxes, fontsize=10,
#                                                  verticalalignment='top', bbox=props)
                                
#                                 # Plot the model loss
#                                 axes[5].plot(history.history['loss'], label='Training Loss')
#                                 axes[5].plot(history.history['val_loss'], label='Validation Loss')
#                                 axes[5].set_title('Model Loss')
#                                 axes[5].set_ylabel('Loss')
#                                 axes[5].set_xlabel('Epoch')
#                                 axes[5].legend(loc='upper right')

#                                 # Adjust layout to prevent overlap
#                                 plt.tight_layout()
                                
#                                 # Ensure x and y axes have the same scale
#                                 for i, ax in enumerate(axes):
#                                     if i < len(output_names):
#                                         ax.set_xlim([min(Y_test[:,i].min(), Y_pred[i].min()), max(Y_test[:,i].max(), Y_pred[i].max())])
#                                         ax.set_ylim([min(Y_test[:,i].min(), Y_pred[i].min()), max(Y_test[:,i].max(), Y_pred[i].max())])
    
#                                 # Save the plot
#                                 plt.savefig(OUTPUT_DIR + f'grid_search_{start_neuron}_{kernel_size}_{strides_size}_{max_pool_size}_{dropout}_{padding}_{batch_size}_{delta}.png')
#                                 plt.close()

# # Save loss history to a text file
# with open(OUTPUT_DIR + 'loss_history.txt', 'w') as f:
#     for entry in loss_history:
#         f.write(f"Configuration: {entry['config']}\n")
#         f.write(f"Training Loss: {entry['training_loss']}\n")
#         f.write(f"Validation Loss: {entry['validation_loss']}\n")
#         f.write(f"{entry['min']}\n")
#         f.write("\n")

# print("Grid search completed and results saved.")

In [ ]:
# param_grid = {
#     'start_neuron': [32,64],
#     'kernel_size': [5,7,9,11],
#     'strides_size': [1,2,3,4],
#     'max_pool_size': [2,3],
#     'dropout': [0.3,0.4,0.5],
#     'padding' :['same','valid'],
#     'batch_size' : [64],
#     'huber_delta' : [0.5,1],
    
# }